In [1]:
using NeuralPDE, Lux, Plots, Random, Distributions, OptimizationOptimJL, OptimizationOptimisers, GLMakie
using IntegralsCubature, FinancialToolbox
using ModelingToolkit: Interval
Random.seed!(0)



TaskLocalRNG()

In [ ]:
using ColorSchemes
#using CairoMakie

In [2]:
#GLMakie.activate!()
#CairoMakie.activate!()
 
σ_min = 0.05
σ_max = 0.15
r_min = 0.05
r_max = 0.15
σ_mid = (σ_min + σ_max)/2
r_mid = (r_min + r_max)/2
K = 100 # strike is 100
T = 1 #one year 
Smin = 80
Smax = 120 # stock range make it 20% in-the-money and out-the-money
scaling_factor = 0.001
boundary_func(s) = max(s - K,0)*scaling_factor

boundary_func (generic function with 1 method)

In [3]:
function call_pricer(x,t,sig,r)
    d1(x,t,sig,r) = (log(x./K) .+ (r .+ 0.5*sig.^2).*(T-t))./(sig*sqrt(T-t))
    d2(x,t,sig,r) = d1.(x,t,sig,r) .- sig*sqrt(T-t)
    return x.*normcdf.(d1.(x,t,sig,r)) .- exp(-r*(T-t))*K*normcdf.(d2.(x,t,sig,r)) 
end

call_pricer (generic function with 1 method)

In [4]:
# 4D PDE in (t, s, σ, r)

@parameters t s σ r
@variables U(..)
Dt = Differential(t)
Ds = Differential(s)
Dss = Differential(s)^2

eq = Dt(U(t, s, σ, r)) + r*s*Ds(U(t, s, σ, r)) + 1/2*σ^2*s^2*Dss(U(t, s, σ, r)) - r*U(t, s, σ, r) ~ 0

# Boundary Conditions
boundary_conditions = [U(T, s, σ, r) ~ boundary_func(s)]

# Problem Domain
domains = [t ∈ Interval(0, T), s ∈ Interval(Smin, Smax), σ ∈ Interval(σ_min, σ_max),         
            r ∈ Interval(r_min, r_max)]

4-element Vector{Symbolics.VarDomainPairing}:
 Symbolics.VarDomainPairing(t, 0..1)
 Symbolics.VarDomainPairing(s, 80..120)
 Symbolics.VarDomainPairing(σ, 0.05..0.15)
 Symbolics.VarDomainPairing(r, 0.05..0.15)

In [5]:
dim = 4
chain = Lux.Chain(Dense(dim, 16, Lux.softplus), Dense(16, 16, Lux.softplus), Dense(16,1)) #Dense(16, 16, Lux.softplus) Dense(16, 1))
ps = Lux.setup(Random.default_rng(), chain)[1]

# Transform PDESystem into OptimizationProblem using PINN methodology
strategy = QuadratureTraining()
#strategy = QuadratureTraining(quadrature_alg = CubatureJLh(),
                     #reltol = 1e-6, abstol = 1e-3,
                     #maxiters = 2_500, batch = 200)

discretization_algo = PhysicsInformedNN(chain, strategy, init_params=ps)
@named pde_system = PDESystem(eq, boundary_conditions, domains, [t, s, σ, r], [U(t, s, σ, r)])
optim_prob = discretize(pde_system, discretization_algo)

OptimizationProblem. In-place: true
u0: ComponentVector{Float32}(layer_1 = (weight = Float32[-0.48351604 0.40789896 0.40354213 0.099549524; -0.049033877 -0.18695378 -0.421895 -0.093530044; … ; -0.22635894 4.831728f-6 0.23992884 -0.3143545; 0.52216566 -0.032581452 -0.35231933 0.07687697], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = Float32[0.15251705 0.39284882 … -0.022368534 0.20915501; -0.2943246 0.08905726 … 0.2225429 -0.1315318; … ; -0.42614084 -0.41936278 … -0.3245835 -0.14442606; 0.047960266 0.27465823 … 0.005657302 0.058415208], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = Float32[0.3952902 0.19821757 … -0.5427456 0.17304797], bias = Float32[0.0;;]))

In [ ]:
#sym_prob = symbolic_discretize(pde_system, discretization_algo)

#pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
#bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions

#callback = function (p, l)
    #println("loss: ", l)
    #println("pde_losses: ", map(l_ -> l_(p), pde_inner_loss_functions))
    #println("bcs_losses: ", map(l_ -> l_(p), bcs_inner_loss_functions))
    #return false
#end

In [ ]:
using TickTock

In [6]:
#tick()
callback = function (p,l)
    println("Current loss is: $l")
    return false
end

opt = BFGS()
result_before = Optimization.solve(optim_prob, opt, callback = callback, maxiters=5000,reltol=1e-6)
result_before.original
phi = discretization_algo.phi


discretization_algo2 = remake(discretization_algo, strategy = strategy , init_params =result_before.u)
prob = discretize(pde_system,discretization_algo2)

opt2 = BFGS()

callback2 = function (p,l)
    println("Current loss2 is: $l")
    return false
end

#pde_inner_loss_functions2 = sym_prob.loss_functions.pde_loss_functions
#bcs_inner_loss_functions2 = sym_prob.loss_functions.bc_loss_functions

#callback2 = function (p,l)
    #println("loss2: ", l)
    #println("pde_losses2: ", map(l_ -> l_(p), pde_inner_loss_functions2))
    #println("bcs_losses2: ", map(l_ -> l_(p), bcs_inner_loss_functions2))
    #return false
#end

result = Optimization.solve(prob, opt, callback = callback2, maxiters=5000, reltol=1e-6)
phi = discretization_algo2.phi

#tock()



Current loss is: 9672.814570056147
Current loss is: 5070.354085337518
Current loss is: 3484.2329231256276
Current loss is: 87.02630716182925
Current loss is: 26.847637602896214
Current loss is: 21.4474708415275
Current loss is: 16.67027478609756
Current loss is: 6.968535052108803
Current loss is: 5.46533931929316
Current loss is: 1.8265346956615685
Current loss is: 0.38348331272733316
Current loss is: 0.07461375921770642
Current loss is: 0.07169065903866209
Current loss is: 0.07161586734336566
Current loss is: 0.07008553328388989
Current loss is: 0.07007847006133819
Current loss is: 0.06914034518108321
Current loss is: 0.036800993692723806
Current loss is: 0.03469856235202583
Current loss is: 0.034528929210694656
Current loss is: 0.03446043651671928
Current loss is: 0.03436388217686509
Current loss is: 0.03435777101272587
Current loss is: 0.03200941728283357
Current loss is: 0.032000311329418114
Current loss is: 0.03187663577504606
Current loss is: 0.03160655359322573
Current loss is: 

Current loss2 is: 0.0002577027294673153
Current loss2 is: 0.00025770169545266436
Current loss2 is: 0.00025770176775400804
Current loss2 is: 0.0002577017605754323


NeuralPDE.Phi{Chain{NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{Dense{true, typeof(NNlib.softplus), typeof(Lux.glorot_uniform), typeof(Lux.zeros32)}, Dense{true, typeof(NNlib.softplus), typeof(Lux.glorot_uniform), typeof(Lux.zeros32)}, Dense{true, typeof(identity), typeof(Lux.glorot_uniform), typeof(Lux.zeros32)}}}}, NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{NamedTuple{(), Tuple{}}, NamedTuple{(), Tuple{}}, NamedTuple{(), Tuple{}}}}}(Chain(), (layer_1 = NamedTuple(), layer_2 = NamedTuple(), layer_3 = NamedTuple()))

In [7]:

 
dx = 0.5
x_domain = Smin:dx:Smax
S_domain = collect(x_domain)
#S_domain = Float32.(S_domain)

times = [0, 0.25, 0.5, 1]
#plots = Array{Plots.Plot{Plots.GRBackend},1}()

u_predict1 = [phi([times[1], current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
u_predict1 = reduce(vcat, u_predict1)/scaling_factor
u_true1 = call_pricer(S_domain, times[1], σ_mid, r_mid)
u_predict2 = [phi([times[2], current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
u_predict2 = reduce(vcat, u_predict2)/scaling_factor
u_true2 = call_pricer(S_domain, times[2], σ_mid, r_mid)
u_predict3 = [phi([times[3], current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
u_predict3 = reduce(vcat, u_predict3)/scaling_factor
u_true3 = call_pricer(S_domain, times[3], σ_mid, r_mid)
u_predict4 = [phi([times[4], current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
u_predict4 = reduce(vcat, u_predict4)/scaling_factor
u_true4 = max.(S_domain .- K, 0)


#u_predict1 = Float32.(u_predict1)
#u_true1 = Float32.(u_true1)

#for i in 1:length(times)
    #current_t = times[i]
    #u_predict = [phi([current_t, current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
    #u_predict = reduce(vcat, u_predict)/scaling_factor
    
   # u_true = call_pricer(S_domain, current_t, σ_mid, r_mid)
   # push!(plots, plot(S_domain, [u_true, u_predict], label= ["Analytical" "PINN"], grid =:false, framestyle =:box, legend=:topleft, title="t = $(current_t)",
            #xlabel = "Stock prices", ylabel = "Option Prices"))
    #push!(plots, plot(S_domain, V_predict, label=""))
#end

#plot(plots[1], plots[2], plots[3], plots[4], layout = 4)

81-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
 14.5
 15.0
 15.5
 16.0
 16.5
 17.0
 17.5
 18.0
 18.5
 19.0
 19.5
 20.0

In [8]:
#figure(1);
using PyPlot
fig = figure(figsize=(10, 10), dpi=1200);
#subplots_adjust(hspace=0.5,wspace=0.9)
subplot(221) ;  PyPlot.plot(x_domain,u_predict1,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true1,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(222) ;  PyPlot.plot(x_domain,u_predict2,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true2,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0.25",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(223) ;  PyPlot.plot(x_domain,u_predict3,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true3,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0.5",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(224) ;  PyPlot.plot(x_domain,u_predict4,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true4,color = "magenta", label = "Analytical");
PyPlot.title("Time = 1",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

fig
save("./Fig_22.png", fig)

## Error plots 

In [ ]:
 #EXample code from Makie


fig = Figure()
Axis(fig[1, 1])

xs = 0:0.01:10
ys = 0.5 .* sin.(xs)

for (i, lw) in enumerate([1, 2, 3])
    lines!(xs, ys .- i/6, linestyle = nothing, linewidth = lw)
    lines!(xs, ys .- i/6 .- 1, linestyle = :dash, linewidth = lw)
    lines!(xs, ys .- i/6 .- 2, linestyle = :dot, linewidth = lw)
    lines!(xs, ys .- i/6 .- 3, linestyle = :dashdot, linewidth = lw)
    lines!(xs, ys .- i/6 .- 4, linestyle = :dashdotdot, linewidth = lw)
    lines!(xs, ys .- i/6 .- 5, linestyle = [0.5, 1.0, 1.5, 2.5], linewidth = lw)
end

fig

In [ ]:

#using LaTeXStrings

fig = Figure(resolution = (800,800),fontsize = 22)

ax1 = Axis(fig[1, 1] ,xgridvisible = :false, ygridvisible = :false, xlabel = L"$S$",
    ylabel = "Call Price")
ax2 = Axis(fig[1, 2], title = "PINN vs Analytical",  xgridvisible = :false, ygridvisible = :false, xlabel = L"S",
    ylabel = "Call price")
ax3 = Axis(fig[2, 1], title = "PINN vs Analytical", xgridvisible = :false, ygridvisible = :false, xlabel = L"S",
    ylabel = "Call price")
ax4 = Axis(fig[2, 2], title = "PINN vs Analytical", xgridvisible = :false, ygridvisible = :false, xlabel = L"S",
    ylabel = "Call price")

ax1.title = "PINN vs Analytical" 

lines!(ax1, S_domain , u_predict1 , color = :red, linewidth = 2.5)
lines!(ax1, S_domain, u_true1, color = :blue,linewidth = 2.5)
#axislegend()
lines!(ax2, S_domain, u_predict2 , color = :red,linewidth = 2.5)
lines!(ax2, S_domain, u_true2, color = :blue, linewidth = 2.5)
#axislegend()
lines!(ax3, S_domain, u_predict3 , color = :red,linewidth = 2.5)  
lines!(ax3, S_domain, u_true3, color = :blue, linewidth = 2.5)
#axislegend()
lines!(ax4, S_domain, u_predict4 , color = :red, linewidth = 2.5)
lines!(fig[2, 2], S_domain, u_true4, color = :blue, linewidth = 2.5)
#save("./Fig_1_1.png", fig)
#axislegend()
fig



In [ ]:
# Error plot 


plots = Array{Plots.Plot{Plots.GRBackend},1}()
for i in 1:length(times)
    current_t = times[i]
    u_predict = [phi([current_t, current_x, σ_mid, r_mid],result.u) for current_x in S_domain]
    u_predict = reduce(vcat, u_predict)/scaling_factor
    u_true = call_pricer(S_domain, current_t, σ_mid, r_mid)
    error = abs.(u_true - u_predict)
    push!(plots, plot(S_domain, error, legend=:topleft, title="Time = $(current_t)"))
    
end

plot(plots[1], plots[2], plots[3], plots[4], layout = 4)

## Code for meshgrid 

In [ ]:
using LaTeXStrings

ts = collect(range(0,T,length=100))
xs = collect(range(Smin,Smax,length=100))
σs = collect(range(σ_min,σ_max,length=100))
rs = collect(range(r_min,r_max,length=100))

meshgrid(x, ys) = [x[i] for i in 1:length(x), j in length(ys)], [ys[j] for j in 1:length(ys), i in 1:length(x)]
domain_x, domain_t = meshgrid(xs,ts)    #use domain_t as meshgrid for time
domain_y2, domain_s = meshgrid(ts,xs)   #use domain_s as meshgrid for stock prices
domain_x, domain_σ = meshgrid(xs,σs)    #use domain_σ as meshgrid for σ range
domain_x, domain_r = meshgrid(xs,rs)    #use domain_r as meshgrid for r range 



## Code for matrix of analytical solutions and PINN

In [ ]:


predict_matrix1 = zeros(length(ts),length(xs))
analytical_matrix1 = zeros(length(ts),length(xs))
for i in 1:length(ts)
    time = ts[i]
    predict1 = [phi([time, stock, σ_mid, r_mid],result.u) for stock in xs]
    predict1 = reduce(vcat, predict1)/scaling_factor
    u_true1 = [call_pricer(ss, time, σ_mid, r_mid) for ss in xs]
    predict_matrix1[:,i] = predict1
    analytical_matrix1[:,i] = u_true1
end

#predict1 = reshape([first(phi([t,s,σ_mid,r_mid],result.u)) for t in ts for s in xs]*scaling_factor,
    #(length(ts)),(length(xs)))
#real_surf = reshape([first(call_pricer.(x,t,σ_mid,r_mid)) for x in xs for t in ts],
    #(length(xs),length(ts)))

## Code for matrix of analytical solutions - letting sigma and stock change


In [ ]:


predict_matrix2 = zeros(length(σs),length(xs))
analytical_matrix2 = zeros(length(σs),length(xs))
for i in 1:length(σs)
    current_σ = σs[i]
    predict2 = [phi([0, stock, current_σ, r_mid],result.u) for stock in xs]
    predict2 = reduce(vcat, predict2)/scaling_factor
    u_true2 = [call_pricer(ss, 0, current_σ, r_mid) for ss in xs]
    predict_matrix2[:,i] = predict2
    analytical_matrix2[:,i] = u_true2
end


## Code for matrix of analytical solutions - letting return and stock change

In [ ]:


predict_matrix3 = zeros(length(rs),length(xs))
analytical_matrix3 = zeros(length(rs),length(xs))
for i in 1:length(rs)
    current_r = rs[i]
    predict3 = [phi([0, stock, σ_mid, current_r],result.u) for stock in xs]
    predict3 = reduce(vcat, predict3)/scaling_factor
    u_true3 = [call_pricer(ss, 0, σ_mid, current_r) for ss in xs]
    predict_matrix3[:,i] = predict3
    analytical_matrix3[:,i] = u_true3
end

In [ ]:
# first plot

 
#using LaTeXStrings

fig = Figure(resolution=(1400, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "t", zlabel = "Call Price",
    title = "PINN")
ax2 = Axis3(fig[1, 2], perspectiveness=0.2, xlabel = "S", ylabel = "t", zlabel = " Call Price",
    title = "Analytical solution")

GLMakie.surface!(ax1, domain_s , domain_t', predict_matrix1 , transparency = true)
GLMakie.surface!(ax2, domain_s, domain_t', analytical_matrix1 , transparency = true)
save("./Fig_1_2.png", fig)
fig

#j1 = plot(domain_s',domain_t,predict_matrix, st = :surface, xlabel = "stock prices", ylabel = "time", 
    #zlabel = "call prices", title = "PINNs solution")


In [ ]:
# Second plot

fig = Figure(resolution=(1200, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "t", zlabel = "Absolute error",
    title = "Absolute error")


GLMakie.surface!(ax1, domain_s , domain_t', abs.(analytical_matrix1 .- predict_matrix1) , transparency = true)

save("./Fig_1_3.png", fig)
fig

In [ ]:

# Third plot

#solar = ColorSchemes.solar.colors
fig = Figure(resolution=(1400, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "σ", zlabel = "Call Price",
    title = "PINN")
ax2 = Axis3(fig[1, 2], perspectiveness=0.2, xlabel = "S", ylabel = "σ", zlabel = " Call Price",
    title = "Analytical")

GLMakie.surface!(ax1, domain_s , domain_σ', predict_matrix2 , transparency = true, colormap = ColorSchemes.cool)
GLMakie.surface!(ax2, domain_s, domain_σ', analytical_matrix2 , transparency = true, colormap = ColorSchemes.cool)
save("./Fig_1_4.png", fig)
fig

In [ ]:
# Fourth plot

fig = Figure(resolution=(1200, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "σ", zlabel = "Absolute error",
    title = "Absolute error")


GLMakie.surface!(ax1, domain_s , domain_σ', abs.(analytical_matrix2 .- predict_matrix2) , transparency = true,
                    colormap = ColorSchemes.cool)

save("./Fig_1_5.png", fig)
fig

In [ ]:
# Fifth plot

#solar = ColorSchemes.solar.colors
fig = Figure(resolution=(1400, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "r", zlabel = "Call Price",
    title = "PINN")
ax2 = Axis3(fig[1, 2], perspectiveness=0.2, xlabel = "S", ylabel = "r", zlabel = " Call Price",
    title = "Analytical")

GLMakie.surface!(ax1, domain_s , domain_r', predict_matrix3 , transparency = true, colormap = ColorSchemes.jet)
GLMakie.surface!(ax2, domain_s, domain_r', analytical_matrix3 , transparency = true, colormap = ColorSchemes.jet)
save("./Fig_1_6.png", fig)
fig

In [ ]:
# sixth plot

fig = Figure(resolution=(1200, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "r", zlabel = "Absolute error",
    title = "Absolute error")


GLMakie.surface!(ax1, domain_s , domain_r', abs.(analytical_matrix3 .- predict_matrix3) , transparency = true, 
            colormap = ColorSchemes.jet)

save("./Fig_1_7.png", fig)
fig

In [ ]:
#j2 = plot(domain_s',domain_t,analytical_matrix, st = :surface, xlabel = "stock prices", ylabel = "time",
    #zlabel = "call prices", title = "Analytical solution")

In [ ]:
error = plot(domain_s',domain_t,abs.(predict_matrix .- analytical_matrix), st = :surface, xlabel = "stock prices", ylabel = "time",
    zlabel = "call prices", title = "Error plot",axis=([], false))


In [ ]:
using Plots
pyplot()

j3 = plot(domain_s',domain_σ,predict_matrix2, st = :surface, xlabel = "stock prices", ylabel = "σ", 
    zlabel = "call prices", title = "PINNs solution")

In [ ]:
j4 = plot(domain_s',domain_σ, analytical_matrix2, st = :surface, xlabel = "stock prices", ylabel = "σ", 
    zlabel = "call prices", title = "Analytical solution")

In [ ]:
j5 = plot(domain_s',domain_σ, abs.(predict_matrix2 .- analytical_matrix2), st = :surface, xlabel = "stock prices", ylabel = "σ", 
    zlabel = "call prices", title = "Error plot")

In [ ]:
j6 = plot(domain_s',domain_r,predict_matrix3, st = :surface, xlabel = "stock prices", ylabel = "r", 
    zlabel = "call prices", title = "PINNs solution")

In [ ]:
j7 = plot(domain_s',domain_r,analytical_matrix3, st = :surface, xlabel = "stock prices", ylabel = "r", 
    zlabel = "call prices", title = "Analytical solution")

In [ ]:
j8 = plot(domain_s',domain_r, abs.(predict_matrix3 .- analytical_matrix3), st = :surface, xlabel = "stock prices", ylabel = "r", 
    zlabel = "call prices", title = "Analytical solution")